In [1]:
import os, json, sys, shutil
from ovejero import model_trainer

# Overfitting a Toy Model Using model_trainer

__Author:__ Sebastian Wagner-Carena

__Created:__ 10/24/2019

__Last Run:__ 10/24/2019

__Goals:__ Learn how to use model_trainer to overfit the three types of models used by ovejero

__Before running this notebook:__ Run the Generate_Config notebook to understand what goes into the configuration files for overjero

We'll start by loading up the test configuration file made by Generate_Config and inspecting it.

In [2]:
json_path = os.getcwd()[:-5]+'test/test_data/' + 'test.json'
with open(json_path,'r') as json_f:
    cfg = json.load(json_f)
print(cfg)

{'training_params': {'bnn_type': 'gmm', 'batch_size': 10, 'n_epochs': 10, 'learning_rate': 0.0001, 'decay': 3e-06, 'kernel_regularizer': 1e-05, 'dropout_regularizer': 1e-06, 'root_path': '/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/', 'tf_record_path': 'tf_record_test', 'final_params': ['external_shear_e1', 'external_shear_e2', 'lens_mass_center_x', 'lens_mass_center_y', 'lens_mass_e1', 'lens_mass_e2', 'lens_mass_gamma', 'lens_mass_theta_E_log'], 'flip_pairs': [[0, 1], [4, 5], [0, 1, 4, 5]], 'img_dim': 128, 'model_weights': '/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/test_model.h5', 'tensorboard_log_dir': '/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/test.log', 'random_seed': 1138}, 'validation_params': {'root_path': '/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/', 'tf_record_path': 'tf_record_test_val'}, 'dataset_params': {'lens_params_path': 'metadata.csv

A lot of good information there! This is a good config file to start with. Let's go ahead and change a few paths and use it for our toy model.

In [3]:
# Change the model weights to point to the demo directory! Same for log file.
print('old path:')
print(cfg['training_params']['model_weights'])

cfg['training_params']['model_weights'] = os.getcwd() + '/test_model.h5'
cfg['training_params']['tensorboard_log_dir'] = os.getcwd() + '/test_logs'
print('new path:')
print(cfg['training_params']['model_weights'])

# Also let's start with the easy diagonal case
cfg['training_params']['bnn_type'] = 'diag'

# Now let's go ahead and save this as our new configuration file
diag_json_path = os.getcwd() + '/diag.json'
with open(diag_json_path,'w') as json_f:
    json.dump(cfg,json_f,indent=4)

old path:
/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/test_model.h5
new path:
/Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/demos/test_model.h5


All we have to do is call the main function of model_trainer with the path to our config file! You should see the loss go down as the model learns to overfit to the lenses in the very small training set. Because the random seed is set by the configuration file the final loss should be 2.8351

In [4]:
# This is equivalent to 'python -m model_trainer diag_json_path' in the terminal where diag_json_path is the path.
sys.argv = ['model_trainer',diag_json_path]
model_trainer.main()

  0%|          | 0/20 [00:00<?, ?it/s]

Checking for training data.
Generating new TFRecord at /Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/tf_record_test
Checking for validation data.
Generating new TFRecord at /Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/test/test_data/tf_record_test_val


100%|██████████| 20/20 [00:00<00:00, 356.91it/s]


Initializing the model
Is model built: True
No weights found. Saving new weights to /Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/demos/test_model.h5
Epoch 1/10
2/2 [==============================] - 9s 5s/step - loss: 4.1263 - diagonal_covariance_loss: 3.9816 - val_loss: 0.0000e+00 - val_diagonal_covariance_loss: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 6s 3s/step - loss: 4.0302 - diagonal_covariance_loss: 3.8051 - val_loss: 3.6275 - val_diagonal_covariance_loss: 3.4831
Epoch 3/10
2/2 [==============================] - 6s 3s/step - loss: 3.3157 - diagonal_covariance_loss: 3.3097 - val_loss: 3.6658 - val_diagonal_covariance_loss: 3.5216
Epoch 4/10
2/2 [==============================] - 6s 3s/step - loss: 4.1434 - diagonal_covariance_loss: 3.5543 - val_loss: 3.3515 - val_diagonal_covariance_loss: 3.2075
Epoch 5/10
2/2 [==============================] - 6s 3s/step - loss: 3.0307 - diagonal_covariance_loss: 3.1071 - val_loss: 3.2585 - val_diagonal_covar

At the end of each epoch we get the loss (which includes the concrete dropout regularization penalty) and the diagonal/full/gmm loss term (essentially the measure of how well our pdf is doing at capturing the data) on both the training and validation sets. 

In [5]:
# Clean up the files that were created by this notebook.
test_data_path = os.getcwd()[:-5]+'test/test_data/'
os.remove(test_data_path+'new_metadata.csv')
os.remove(test_data_path+'norms.npy')
os.remove(test_data_path+'tf_record_test')
os.remove(test_data_path+'tf_record_test_val')
os.remove('test_model.h5')
shutil.rmtree('test_logs')
os.remove('diag.json')